In [57]:
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
from transformers import logging
from torch.nn.functional import cosine_similarity
from pprint import pprint

In [27]:
 ログレベルを設定（この例ではWARNINGレベル以上のみ表示）
logging.set_verbosity_warning()

SyntaxError: invalid character '（' (U+FF08) (2627790753.py, line 1)

In [28]:
# llm-book/bert-base-japanes-v3-marc_jaを使用する
text_classification_pipline = pipeline(
    model="llm-book/bert-base-japanese-v3-marc_ja"
)

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--llm-book--bert-base-japanese-v3-marc_ja/snapshots/7b47edf80477fd9da0ee1bc1908326ac012d624f/config.json
Model config BertConfig {
  "_name_or_path": "llm-book/bert-base-japanese-v3-marc_ja",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "positive",
    "1": "negative"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "negative": 1,
    "positive": 0
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "torch_dtype": "float32",
  "transformers_version": "4.39.1",
  "type_vocab_size"

In [8]:
# positiveの予想
positive_text = "世界には言葉がわからなくても感動する音楽がある。"
print(text_classification_pipline(positive_text)[0])

{'label': 'positive', 'score': 0.9993619322776794}


In [18]:
# negative

nagative_text = 'アンニュイ'
print(text_classification_pipline(nagative_text)[0])

{'label': 'positive', 'score': 0.9981855750083923}


## 自然言語推論(NLI)  

・二つのテキストの論理関係を予測するタスク  
・言語モデルの意味理解力を評価するために使用

In [29]:
nli_pipeline = pipeline(model="llm-book/bert-base-japanese-v3-jnli", device='cuda:0')
text = "二人の男性がジェット機を見ています"
entailment_text = "ジェット機を見ている人が二人います"

print(nli_pipeline({'text':text, 'text_pair':entailment_text}))

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--llm-book--bert-base-japanese-v3-jnli/snapshots/9056fce079ed3fc284c9b2d1c2abccae3d13af61/config.json
Model config BertConfig {
  "_name_or_path": "llm-book/bert-base-japanese-v3-jnli",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "entailment",
    "1": "contradiction",
    "2": "neutral"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "contradiction": 1,
    "entailment": 0,
    "neutral": 2
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "torch_dtype": "float32",
  "transf

{'label': 'entailment', 'score': 0.9964311122894287}


In [22]:
contradiction_text = "二人の男性が飛んでいます"
# textとcontradiction_textの論理関係を予測
print(nli_pipeline({"text": text, "text_pair": contradiction_text}))

{'label': 'contradiction', 'score': 0.9990535378456116}


In [23]:
netral_text = '二人の男性が、白い飛行機を眺めています'
print(nli_pipeline({'text':text, 'text_pair':netral_text}))

{'label': 'neutral', 'score': 0.9579205513000488}


## 意味的類似度の計算  
  
・二つのテキストの意味が似ている度合いをスコアとしてよそくするタスク。  
・情報検索や、複数テキストの内容の整合性を確認する際に役立ちます。  
・0~5の値が返る

In [35]:
logging.set_verbosity_error()
text_sim_pipeline = pipeline(
    model="llm-book/bert-base-japanese-v3-jsts",
    function_to_apply="none",
)
text = "川べりでサーフボードを持った人たちがいます"
sim_text = "サーファーたちが川べりに立っています"
# textとsim_textの類似度を計算
result = text_sim_pipeline({"text": text, "text_pair": sim_text})
print(result["score"])

3.5703563690185547


In [36]:
dissim_text = "トイレの壁に黒いタオルがかけられています"
# textとdissim_textの類似度を計算
result = text_sim_pipeline({"text": text, "text_pair": dissim_text})
print(result["score"])

0.041621580719947815


In [39]:
# コサイン類似度を使用した意味的類似度
sim_enc_pipeline = pipeline(
    model="llm-book/bert-base-japanese-v3-unsup-simcse-jawiki",
    task="feature-extraction",
)

config.json:   0%|          | 0.00/634 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/445M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/529 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/231k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [48]:
# ベクトルを獲得
text_emb = sim_enc_pipeline(text, return_tensors=True)[0][0]
sim_emb = sim_enc_pipeline(sim_text, return_tensors=True)[0][0]

# コサイン類似度
sim_pair_score = cosine_similarity(text_emb, sim_emb, dim=0)
print(sim_pair_score.item())

0.8568588495254517


In [47]:
# dissim_textのベクトルを獲得
dissim_emb = sim_enc_pipeline(dissim_text, return_tensors=True)[0][0]
# textとdissim_textの類似度を計算
dissim_pair_score = cosine_similarity(text_emb, dissim_emb, dim=0)
print(dissim_pair_score.item())

0.45887044072151184


## 固有表現認識（NER）  
・テキストに含まれる固有表現を抽出

In [51]:
from pprint import pprint

ner_pipeline = pipeline(
    model="llm-book/bert-base-japanese-v3-ner-wikipedia-dataset",
    aggregation_strategy="simple",
)
text = "大谷翔平は岩手県水沢市出身のプロ野球選手"
# text中の固有表現を抽出
pprint(ner_pipeline(text))


config.json:   0%|          | 0.00/1.93k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/443M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/529 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/231k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

[{'end': None,
  'entity_group': '人名',
  'score': 0.99823624,
  'start': None,
  'word': '大谷 翔平'},
 {'end': None,
  'entity_group': '地名',
  'score': 0.9986874,
  'start': None,
  'word': '岩手 県 水沢 市'}]


## 要約生成

In [53]:
text2text_pipeline = pipeline(
    model="llm-book/t5-base-long-livedoor-news-corpus"
)
article = "ついに始まった３連休。テレビを見ながら過ごしている人も多いのではないだろうか？　今夜オススメなのは何と言っても、NHKスペシャル「世界を変えた男 スティーブ・ジョブズ」だ。実は知らない人も多いジョブズ氏の養子に出された生い立ちや、アップル社から一時追放されるなどの経験。そして、彼が追い求めた理想の未来とはなんだったのか、ファンならずとも気になる内容になっている。 今年、亡くなったジョブズ氏の伝記は日本でもベストセラーになっている。今後もアップル製品だけでなく、世界でのジョブズ氏の影響は大きいだろうと想像される。ジョブズ氏のことをあまり知らないという人もこの機会にぜひチェックしてみよう。 世界を変えた男　スティーブ・ジョブズ（NHKスペシャル）"
# articleの要約を生成
print(text2text_pipeline(article)[0]["generated_text"])

config.json:   0%|          | 0.00/768 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/142 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.38k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/798k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.43M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1132: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


今夜はNHKスペシャル「世界を変えた男 スティーブ・ジョブズ」をチェック!


In [56]:
# tokenizerのロード
tokenizer = AutoTokenizer.from_pretrained('abeja/gpt2-large-japanese')

# テキストのトークン化
tokenizer.tokenize('今日は天気が良いので')

tokenizer_config.json:   0%|          | 0.00/282 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/784k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/153 [00:00<?, ?B/s]

['▁', '今日', 'は', '天気', 'が良い', 'の', 'で']

In [61]:
model = AutoModelForCausalLM.from_pretrained('abeja/gpt2-large-japanese')

# トークン化 -> ベクトル化
input = tokenizer('今日は天気が良いので', return_tensors='pt')

outputs = model.generate(
**input,
    max_length=15,
    pad_token_id=tokenizer.pad_token_id)

generated_text = tokenizer.decode(
    outputs[0], skip_special_tokens=True
)

print(generated_text)

今日は天気が良いので外でお弁当を食べました。


In [67]:
input

{'input_ids': tensor([[    7,  4639,    15, 17110, 16958,    10,    20,     2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1]])}